In [1]:
! pip list | grep langchain
! pip list | grep opensearch

langchain                     0.0.330
opensearch-py                 2.3.2


In [2]:
# %load_ext autoreload
%reload_ext autoreload
%autoreload 2

import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))
print(sys.path)
from utils import print_ww

['/home/whpark/workshop/bedrock/amazon-bedrock-workshop-webinar-kr/01_Generation', '/home/whpark/miniconda3/envs/bedrock/lib/python310.zip', '/home/whpark/miniconda3/envs/bedrock/lib/python3.10', '/home/whpark/miniconda3/envs/bedrock/lib/python3.10/lib-dynload', '', '/home/whpark/miniconda3/envs/bedrock/lib/python3.10/site-packages', '/home/whpark/workshop/bedrock/amazon-bedrock-workshop-webinar-kr']


In [3]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

os.environ["AWS_DEFAULT_REGION"] = "us-east-1"  # E.g. "us-east-1"
os.environ["BEDROCK_ASSUME_ROLE"] = "arn:aws:iam::532805286864:role/bedrock_vscode"
# os.environ["AWS_PROFILE"] = "default"

boto3_bedrock = bedrock.get_bedrock_client(
  assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
  endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
  region=os.environ.get("AWS_DEFAULT_REGION", None)
)

print(colored("\n\n== FM lists ==", "green"))
pprint(bedrock_info.get_list_fm_models())

Create new client
  Using region: us-east-1
  Using profile: None
  Using role: arn:aws:iam::532805286864:role/bedrock_vscode ... successful!
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Command': 'cohere.command-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'TBD'}


### Generate text

In [4]:
# create the prompt
prompt_data = """
Command: Write an email from Bob, Customer Service Manager, to the customer "John Doe"
who provided negative feedback on the service provided by our customer support engineer
 """

In [5]:
body = json.dumps({
  "inputText": prompt_data,
  "textGenerationConfig": {
    "maxTokenCount": 4096,
    "stopSequence": [],
    "temperature": 0,
    "topP": 0.9
  }
})

In [7]:
modelId = 'amazon.titan-tg1-large'
accept = 'application/json'
contentType = 'application/json'

response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

outputText = response_body.get('results')[0].get('outputText')

AccessDeniedException: An error occurred (AccessDeniedException) when calling the InvokeModel operation: Your account is not authorized to invoke this API operation.